# QSR Menu Chatbot Demo

Welcome to this tutorial! We'll explore how to use generative AI, specifically large language models (LLMs), to interact with a Quick Service Restaurant (QSR) menu stored in a SQL database. Our goal is to enable users to ask natural language questions about the menu (e.g., "What vegetarian options are there?", "What's the cheapest burger?") and receive informative answers. We'll use LangChain to simplify the connection between a Large Language Model (LLM) and a SQL database containing menu items.

Prerequisites:
- Basic knowledge of Python and LLMs
- API access to an OpenAI-compatible LLM (examples provided for locally hosted Ollama and NVIDIA-hosted models)

In a follow-up demo, we'll explore a more customizable setup using LangGraph.

## Install Dependencies

In [2]:
try:
    import sqlalchemy
except ImportError:
    %pip install sqlalchemy -q
    import sqlalchemy

# LangChain-related packages
try:
    from langchain_openai import ChatOpenAI
except ImportError:
    %pip install langchain-openai -q
    from langchain_openai import ChatOpenAI

try:
    from langchain_community.tools.sql_database.tool import SQLDatabase
    from langchain.agents.agent_toolkits import SQLDatabaseToolkit
    from langchain.agents import create_sql_agent, AgentType
except ImportError:
    %pip install langchain-community -q
    from langchain_community.tools.sql_database.tool import SQLDatabase
    from langchain.agents.agent_toolkits import SQLDatabaseToolkit
    from langchain.agents import create_sql_agent, AgentType

## Setup Example Menu Database

Now that we have our libraries, let's create the data source our agent will interact with: a SQLite database representing a QSR menu.

The code cell below uses a helper function `build_qsr_menu_database` (presumably located in a `utils` directory) which:
1.  Connects to (or creates) a SQLite database file (e.g., `qsr_menu.db`).
2.  Defines the necessary tables (like `menu`, `nutrition_facts`) using SQLAlchemy's ORM.
3.  Populates these tables with sample menu data loaded from a file (e.g., a JSON file in a `data` directory).
4.  Returns a SQLAlchemy `Engine` object, which represents the connection pool to our database.

After creating the database engine, we wrap it in LangChain's `SQLDatabase` class. This is a crucial step because the `SQLDatabase` object provides convenient methods for our agent's tools, such as:
* `get_table_info`: To retrieve the schema (column names, types, keys) of specified tables.
* `get_usable_table_names`: To list the available tables.
* `run` or `run_no_throw`: To execute SQL queries against the database.

This abstraction simplifies how the agent interacts with the database, allowing it to focus on generating the *correct* SQL rather than managing database connections.

In [4]:
from utils.qsr_db_builder import build_qsr_menu_database

engine = build_qsr_menu_database("./data/menu.json", "sqlite:///./data/qsr_menu.db")

# SQLDatabase object for LangChain
db = SQLDatabase(engine)

Database created and populated from ./data/menu.json


## Initialize LangChain Agent
Now, we'll connect to a local Ollama LLM and initialize an agent that can intelligently query the QSR menu database. This example assumes an Ollama server is currently hosted at IP address 192.168.1.23 and it already has the following model installed: llama3.3:70b-instruct-q3_K_S.

Here we are using a community-developed SQL agent that is part of the LangChain community library. This is an easy way to get up and running with a SQL agent with minimal work.

In [5]:
# Use the line below to use the Ollama model running at ip address 192.168.1.23. Change this to your own Ollama server IP address.
llm = ChatOpenAI(base_url="http://192.168.1.23:11434/v1", model="llama3.3:70b-instruct-q3_K_S", api_key="ollama", temperature=0.0)

# Setup SQL toolkit and LangChain agent
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

## Experiment with Queries
Now try sending queries to your SQL agent. Try differnt levels of prompt complexity to test the limits of your LLM.

In [8]:
# Natural language query example
input_data = {"input": "What is the healthiest meal (not Drink) on the menu?"}
response_dict = agent_executor.invoke(input_data)

# Print the response
print(response_dict.get('output', response_dict))



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: menu, nutrition_factsI have the list of tables in the database, which are "menu" and "nutrition_facts". Now, I should query the schema of these two tables to see what columns they contain.

Action: sql_db_schema
Action Input: menu, nutrition_facts
CREATE TABLE menu (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	category VARCHAR NOT NULL, 
	price FLOAT NOT NULL, 
	ingredients VARCHAR, 
	PRIMARY KEY (id)
)

/*
3 rows from menu table:
id	name	category	price	ingredients
1	Classic Cheeseburger	Entree	5.99	Beef patty, cheese, lettuce, tomato, pickles, bun, condiments
2	Bacon Cheeseburger	Entree	6.99	Beef patty, bacon, cheese, lettuce, tomato, onions, bun, condiments
3	Double Deluxe Burger	Entree	7.99	Two beef patties, double cheese, lettuce, tomato, pickles, onions, special sauce, bun
*/


CREATE TABLE nutrition_facts (
	item_id INTEGER NOT NULL, 
	calories INTEGER, 
	protein_g FLOAT, 
	fat_g FLOAT, 
	c

## Use NVIDIA-hosted Llama 3.3 70b Model
Now, let's demonstrate querying the same database using a powerful NVIDIA-hosted model (Llama 3.3 70b). You'll need an API key to proceed.

In [ ]:
# Enter your NVIDIA API key
api_key = "your-api-key-here"

# Connect to NVIDIA-hosted Llama 3.3 model
llm = ChatOpenAI(base_url="https://integrate.api.nvidia.com/v1", model="meta/llama-3.3-70b-instruct", api_key=api_key)

# Re-initialize SQL toolkit and agent for NVIDIA model
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

Run a query to compare the NVIDIA model's response:

In [ ]:
# Query using NVIDIA-hosted LLM
response = agent_executor.run("What is the healthiest meal (not Drink) on the menu?")
print(response)